In [1]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
from astropy.cosmology import default_cosmology

from lenstronomy.Data.pixel_grid import PixelGrid
from lenstronomy.Data.psf import PSF
from lenstronomy.ImSim.image_model import ImageModel
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.SimulationAPI.ObservationConfig import Roman
from lenstronomy.Util import data_util

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.lenses import lens_util
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import lenstronomy_sim, survey_sim

In [2]:
pickles = glob(f'{config.machine.dir_00}/detectable_gglenses_*.pkl')
detectable_gglenses = []

for p in tqdm(pickles):
    gglenses = util.unpickle(p)
    detectable_gglenses.extend(gglenses)

print(f'{len(detectable_gglenses)} detectable lens(es)')

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:57<00:00,  1.61s/it]

235 detectable lens(es)


In [6]:
# limit
detectable_gglenses = detectable_gglenses[:25]

In [7]:
figure_output_dir = os.path.join(data_dir, 'snr_estimate')
util.create_directory_if_not_exists(figure_output_dir)
util.clear_directory(figure_output_dir)

In [9]:
for band in ['F106', 'F129', 'F184']:
    for i, gglens in tqdm(enumerate(detectable_gglenses), total=len(detectable_gglenses)):
        total_image, lens_surface_brightness, source_surface_brightness, sim_r = survey_sim.get_image(gglens, band)

        snr, total_image_with_noise = survey_sim.get_snr(gglens, band, mask_mult=1, zodi_mult=1.5, total_image=total_image, source_surface_brightness=source_surface_brightness, sim_api=sim_r)

        f, ax = plt.subplots(1, 5, figsize=(20, 4))

        ax[0].imshow(source_surface_brightness)
        ax[1].imshow(lens_surface_brightness)
        ax[2].imshow(total_image_with_noise - total_image)
        ax[3].imshow(np.log10(total_image))
        ax[4].imshow(total_image_with_noise)

        ax[0].set_title('Source Surface Brightness')
        ax[1].set_title('Lens Surface Brightness')
        ax[2].set_title('Noise')
        ax[3].set_title('Total Image (log10)')
        ax[4].set_title('Total Image with Noise')
        plt.suptitle(f'SNR: {snr}')

        plt.savefig(f'{figure_output_dir}/{str(i).zfill(4)}_{band}_snr_estimate.jpg')
        plt.close()

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [01:11<00:00,  2.87s/it]
